# 1 perpare AKS

## load environment variables

change ${PJ_ROOT} to your directory.

In [ ]:
export PJ_ROOT="${HOME}/core"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/core
```

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## login AKS

In [ ]:
az login --tenant ${TENANT}

## create DNS zone

In [ ]:
az group create --name ${DNS_ZONE_RG} --location ${REGION}

In [ ]:
az network dns zone create --resource-group ${DNS_ZONE_RG} --name "${DOMAIN}"

In [ ]:
az network dns zone show --resource-group ${DNS_ZONE_RG} --name "${DOMAIN}" | jq ".nameServers"

example)
```
[
  "ns1-XX.azure-dns.com.",
  "ns2-XX.azure-dns.net.",
  "ns3-XX.azure-dns.org.",
  "ns4-XX.azure-dns.info."
]
```

## create Resource Group for AKS

In [ ]:
az group create --name ${AKS_RG} --location ${REGION}

## start private registry on Azure Container Registry

In [ ]:
az acr create --resource-group ${AKS_RG} --name ${ACR_NAME} --sku Basic

#### for macOS

In [ ]:
export REPOSITORY=$(az acr show --resource-group ${AKS_RG} --name ${ACR_NAME} | jq '.loginServer' -r)
sed -i '' -e "s/<<REPOSITORY>>/${REPOSITORY}/" ${PJ_ROOT}/docs/environments/azure_aks/env
echo ${REPOSITORY}

example)
```
rbcacr.azurecr.io
```

#### for Ubuntu

In [ ]:
export REPOSITORY=$(az acr show --resource-group ${AKS_RG} --name ${ACR_NAME} | jq '.loginServer' -r)
sed -i -e "s/<<REPOSITORY>>/${REPOSITORY}/" ${PJ_ROOT}/docs/environments/azure_aks/env
echo ${REPOSITORY}

example)
```
rbcacr.azurecr.io
```

## start kubernetes on Azure AKS

### prepare AKS

In [ ]:
az provider register -n Microsoft.Compute

In [ ]:
az provider register -n Microsoft.Storage

In [ ]:
az provider register -n Microsoft.Network

In [ ]:
az provider register -n Microsoft.ContainerService

In [ ]:
az provider show -n Microsoft.Compute | jq '.registrationState' -r

expected)
```
Registered
```

In [ ]:
az provider show -n Microsoft.Storage | jq '.registrationState' -r

expected)
```
Registered
```

In [ ]:
az provider show -n Microsoft.Network | jq '.registrationState' -r

expected)
```
Registered
```

In [ ]:
az provider show -n Microsoft.ContainerService | jq '.registrationState' -r

expected)
```
Registered
```

### start AKS

**caution** `az aks create` command takes a long time to execute (about 10~20 minutes).

In [ ]:
export NODE_VM_SIZE="Standard_D2s_v3"
export NODE_OSDISK_SIZE_GB=64
export AKS_VERSION="1.12.7"
az aks create --resource-group ${AKS_RG} --name ${AKS_NAME} --node-count ${NODE_COUNT} --node-vm-size ${NODE_VM_SIZE} --node-osdisk-size ${NODE_OSDISK_SIZE_GB} --ssh-key-value ${SSH_KEY} --kubernetes-version ${AKS_VERSION}

### get credential of AKS

In [ ]:
az aks get-credentials --resource-group ${AKS_RG} --name ${AKS_NAME} --overwrite-existing

In [ ]:
CLIENT_ID=$(az aks show --resource-group ${AKS_RG} --name ${AKS_NAME} --query "servicePrincipalProfile.clientId" --output tsv)
ACR_ID=$(az acr show --resource-group ${AKS_RG} --name ${ACR_NAME} --query "id" --output tsv)
az role assignment create --assignee ${CLIENT_ID} --role Reader --scope ${ACR_ID}

### confirm nodes

In [ ]:
kubectl get nodes

example)
```
NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-41880856-0   Ready    agent   2m20s   v1.12.7
aks-nodepool1-41880856-1   Ready    agent   2m26s   v1.12.7
aks-nodepool1-41880856-2   Ready    agent   2m35s   v1.12.7
aks-nodepool1-41880856-3   Ready    agent   2m8s    v1.12.7
```

### setup rbac

In [ ]:
kubectl apply -f rbac/dashboard-rbac.yaml

In [ ]:
kubectl get serviceaccounts -n kube-system | grep kubernetes-dashboard

expected)
```
kubernetes-dashboard                 1         1m
```

In [ ]:
kubectl apply -f rbac/tiller-rbac.yaml

In [ ]:
kubectl get serviceaccounts -n kube-system | grep tiller

expected)
```
tiller                               1         1m
```

In [ ]:
kubectl apply -f rbac/default-rbac.yaml

In [ ]:
kubectl get clusterroles | grep default-read

expected)
```
default-read                                                           1m
```

In [ ]:
kubectl get clusterrolebindings | grep default

expected)
```
default                                                2m
```

## setup helm

In [ ]:
helm version --client

expected)
```
Client: &version.Version{SemVer:"v2.13.1", GitCommit:"618447cbf203d147601b4b9bd7f8c37a5d39fbb4", GitTreeState:"clean"}
```

In [ ]:
helm init --service-account tiller

In [ ]:
helm repo update

In [ ]:
kubectl get pods --all-namespaces | grep tiller

expected)
```
kube-system   tiller-deploy-7ccf99cd64-gkn7t          1/1       Running   0          1m
```

In [ ]:
helm version

expected)
```
Client: &version.Version{SemVer:"v2.13.1", GitCommit:"618447cbf203d147601b4b9bd7f8c37a5d39fbb4", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.13.1", GitCommit:"618447cbf203d147601b4b9bd7f8c37a5d39fbb4", GitTreeState:"clean"}
```